In [ ]:
%pip install -U azure-ai-ml

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import InteractiveBrowserCredential
from azure.ai.ml.entities import MarketplaceSubscription, ServerlessEndpoint

client = MLClient(
    credential=InteractiveBrowserCredential(tenant_id="<tenant-id>"),
    subscription_id="<subscription-id>",
    resource_group_name="<resource-group-name>",
    workspace_name="<workspace-name>",
)

In [ ]:
# Do this if needed, does not work currently
# but llama3 example works from here - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-serverless?tabs=python

model_id="azureml://registries/azureml-meta/models/Llama-2-7b"
subscription_name="Llama-2-7b"

marketplace_subscription = MarketplaceSubscription(
    model_id=model_id,
    name=subscription_name,
)

marketplace_subscription = client.marketplace_subscriptions.begin_create_or_update(
    marketplace_subscription
).result()


In [ ]:
# Deploy the model as a serverless endpoint
endpoint_name="meta-llama2-7bb-qwerty" # Name must be unique
model_id = "azureml://locations/westus3/workspaces/<workspace_id>/models/default-registered-model-name/versions/2"

In [ ]:
serverless_endpoint = ServerlessEndpoint(
    name=endpoint_name,
    model_id=model_id
)

created_endpoint = client.serverless_endpoints.begin_create_or_update(
    serverless_endpoint
).result()

In [ ]:
endpoint = client.serverless_endpoints.get(endpoint_name)

In [ ]:
import requests

url = f"{endpoint.scoring_uri}/v1/completions"

payload = {
    "max_tokens": 1024,
    "prompt": [
        "Summarize the dialog.\n<dialog>: John: do you play tennis? :-)\r\nIrma: i don't, why do you ask?" +
        "\r\nJohn: you look like someone who plays tennis\r\nIrma: HAHAHAHA i'm literally loling "+
        "\r\nIrma: i don't know what someone who plays tennis looks like\r\nJohn: hahah sorry that was a dumb comment" +
        "\r\nJohn: anyway, would to enroll in a class with me?\r\nJohn: they're dirt cheap at the community center" +
        "\r\nIrma: sounds good, i can wear one of my tennis outfits!!"+
        "\r\nIrma: HAHAHA, i'm still laughing at your comment  lol\n<summary>: "
    ]
}
headers = {
    "Content-Type": "application/json",
    "Authorization": "your-api-key"
}

response = requests.post(url, json=payload, headers=headers)

response.json()